In [1]:
!pip install bertviz
!pip install sacremoses
!pip install wandb -qqq
!pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.6/157.6 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 85.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 93.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 11.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 20.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filena

In [2]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import pandas as pd #yet another array standard
from sklearn.metrics import accuracy_score

from simpletransformers.classification import ClassificationModel, ClassificationArgs

import wandb
wandb.login()

from wandb.keras import WandbCallback

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


# Przygotowanie i wstępna analiza danych

In [3]:
X = np.loadtxt("training_set_clean_only_text.txt", dtype=np.unicode_, delimiter='\n', comments=None)
y = np.loadtxt("training_set_clean_only_tags.txt", delimiter='\n')

df = pd.DataFrame({"text": X, "class": y})

y = np.reshape(y, (-1, 1))

X_test = np.loadtxt("test_set_clean_only_text.txt", dtype=np.unicode_, delimiter='\n', comments=None)
y_test = np.loadtxt("test_set_clean_only_tags.txt", delimiter='\n')

df_test = pd.DataFrame({"text": X_test, "class": y_test})

y_test = np.reshape(y_test, (-1, 1))

In [4]:
positive = 0
for tag in y:
  if tag == 1:
    positive += 1
print("obserwacje treningowe: ", y.size)
print("pozytywne: {}\t({:.3f})".format(positive, positive/y.size))
print("negatywne: {}\t({:.3f})".format(y.size-positive, (y.size-positive)/y.size))

obserwacje treningowe:  10041
pozytywne: 851	(0.085)
negatywne: 9190	(0.915)


In [5]:
val_train_split = 1/3 

X_val = X[:int(X.size*val_train_split)]
y_val = y[:int(y.size*val_train_split)]
X_train = X[int(X.size*val_train_split):]
y_train = y[int(y.size*val_train_split):]

# BERT

In [62]:
bert_args = ClassificationArgs()
bert_args.num_train_epochs = 2
bert_args.learning_rate = 1e-4

In [63]:
modelBert = ClassificationModel("herbert", "allegro/herbert-base-cased",num_labels = 2,args=bert_args, use_cuda=False)

Some weights of the model checkpoint at allegro/herbert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.sso.sso_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.sso.sso_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification 

In [68]:
modelBert.train_model(df)

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:612: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/10041 [00:00<?, ?it/s]

Process ForkPoolWorker-17:
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.10/multiprocessing/queues.py", line 365, in get
    res = self._reader.recv_bytes()
  File "/usr/lib/python3.10/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/usr/lib/python3.10/multiprocessing/connection.py", line 414, in _recv_bytes
    buf = self._recv(4)
  File "/usr/lib/python3.10/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)


KeyboardInterrupt: ignored

In [ ]:
result, model_outputs, wrong_predictions = modelBert.eval_model(df_test, acc=accuracy_score)

# Model keras

In [6]:
vectorizer = keras.layers.TextVectorization()
vectorizer.adapt(X)

In [7]:
vocab = np.array(vectorizer.get_vocabulary())
vectorizer.vocabulary_size()

24062

In [8]:


model = keras.Sequential()
model.add(vectorizer)
model.add(keras.layers.Embedding(vectorizer.vocabulary_size(), 32, mask_zero=True))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 32)          769984    
                                                                 
 global_average_pooling1d (G  (None, 32)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 32)                1056      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 771,073
Trainable params: 771,073
Non-trai

## definicja miar

In [9]:
def recall_metric(y_true, y_pred):
    true_positives = keras.backend.sum(keras.backend.round(keras.backend.clip(y_true * y_pred, 0, 1)))
    possible_positives = keras.backend.sum(keras.backend.round(keras.backend.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + keras.backend.epsilon())
    return recall

def precision_metric(y_true, y_pred):
    true_positives = keras.backend.sum(keras.backend.round(keras.backend.clip(y_true * y_pred, 0, 1)))
    predicted_positives = keras.backend.sum(keras.backend.round(keras.backend.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + keras.backend.epsilon())
    return precision

def f1_metric(y_true, y_pred):
    precision = precision_metric(y_true, y_pred)
    recall = recall_metric(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+keras.backend.epsilon()))

In [10]:
lr = 0.0001
epoch_num = 13
opt = keras.optimizers.Adam(learning_rate=lr)
model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['accuracy', recall_metric, precision_metric, f1_metric])

In [11]:
wandb.init(
project="proj_inl", 
name=f"experiment_lr_{lr}_epoch_{epoch_num}", 
config={
  "learning_rate": lr,
  "architecture": "keras",
  "dataset": "twitter-hate-speech",
  "epochs": epoch_num})
logging_callback = WandbCallback(log_evaluation=True)

wandb: Currently logged in as: s29663 (team29663). Use `wandb login --relogin` to force relogin


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


In [12]:
history = model.fit(X, y, epochs=epoch_num,
                    validation_split=0.2,
                    validation_steps=30,
                    class_weight={0: 1., 1: 10.},
                    callbacks=[logging_callback])

wandb: WARNING WandbCallback is unable to read validation_data from trainer and therefore cannot log validation data. Ensure Keras is properly patched by calling `from wandb.keras import WandbCallback` at the top of your script.


Epoch 1/13
249/251 [============================>.] - ETA: 0s - loss: 1.0837 - accuracy: 0.8100 - recall_metric: 0.0890 - precision_metric: 0.0259 - f1_metric: 0.0256

wandb: ERROR Can't save model in the h5py format. The model will be saved as as an W&B Artifact in the 'tf' format.
wandb: Adding directory to artifact (/content/wandb/run-20230607_105635-5mo2ab12/files/model-best)... Done. 0.1s


251/251 [==============================] - 14s 39ms/step - loss: 1.0837 - accuracy: 0.8110 - recall_metric: 0.0882 - precision_metric: 0.0257 - f1_metric: 0.0254 - val_loss: 0.6762 - val_accuracy: 0.8288 - val_recall_metric: 0.0000e+00 - val_precision_metric: 0.0000e+00 - val_f1_metric: 0.0000e+00
Epoch 2/13
250/251 [============================>.] - ETA: 0s - loss: 1.0688 - accuracy: 0.9369 - recall_metric: 0.0000e+00 - precision_metric: 0.0000e+00 - f1_metric: 0.0000e+00

wandb: ERROR Can't save model in the h5py format. The model will be saved as as an W&B Artifact in the 'tf' format.
wandb: Adding directory to artifact (/content/wandb/run-20230607_105635-5mo2ab12/files/model-best)... Done. 0.0s


...

251/251 [==============================] - 6s 24ms/step - loss: 0.3998 - accuracy: 0.9770 - recall_metric: 0.8576 - precision_metric: 0.6999 - f1_metric: 0.7530 - val_loss: 0.4099 - val_accuracy: 0.8223 - val_recall_metric: 0.2561 - val_precision_metric: 0.4805 - val_f1_metric: 0.3171
Epoch 12/13
249/251 [============================>.] - ETA: 0s - loss: 0.3362 - accuracy: 0.9774 - recall_metric: 0.8394 - precision_metric: 0.6949 - f1_metric: 0.7434

wandb: ERROR Can't save model in the h5py format. The model will be saved as as an W&B Artifact in the 'tf' format.
wandb: Adding directory to artifact (/content/wandb/run-20230607_105635-5mo2ab12/files/model-best)... Done. 0.0s


251/251 [==============================] - 4s 17ms/step - loss: 0.3354 - accuracy: 0.9775 - recall_metric: 0.8406 - precision_metric: 0.6960 - f1_metric: 0.7447 - val_loss: 0.4044 - val_accuracy: 0.8243 - val_recall_metric: 0.2751 - val_precision_metric: 0.4918 - val_f1_metric: 0.3358
Epoch 13/13
248/251 [============================>.] - ETA: 0s - loss: 0.2823 - accuracy: 0.9793 - recall_metric: 0.8930 - precision_metric: 0.7252 - f1_metric: 0.7810

wandb: ERROR Can't save model in the h5py format. The model will be saved as as an W&B Artifact in the 'tf' format.
wandb: Adding directory to artifact (/content/wandb/run-20230607_105635-5mo2ab12/files/model-best)... Done. 0.1s


251/251 [==============================] - 5s 19ms/step - loss: 0.2825 - accuracy: 0.9790 - recall_metric: 0.8890 - precision_metric: 0.7212 - f1_metric: 0.7771 - val_loss: 0.3998 - val_accuracy: 0.8253 - val_recall_metric: 0.2667 - val_precision_metric: 0.4834 - val_f1_metric: 0.3283


# Ewaluacja

In [16]:
test_loss, test_acc, test_rec, test_prec, test_f = model.evaluate(X_test, y_test)

print('Test Loss:\t', test_loss)
print('Test Accuracy:\t', test_acc)
print('Test Precision:\t', test_prec)
print('Test Recall:\t', test_rec)
print('Test F-measure:\t', test_f)

32/32 [==============================] - 0s 3ms/step - loss: 0.3540 - accuracy: 0.8650 - recall_metric: 0.4190 - precision_metric: 0.4505 - f1_metric: 0.3923
Test Loss:	 0.35396909713745117
Test Accuracy:	 0.8650000095367432
Test Precision:	 0.4505208432674408
Test Recall:	 0.41899803280830383
Test F-measure:	 0.3922925591468811
